# 6.4 Interacting with Databases(与数据库的交互)

如果在工作中，大部分数据并不会以text或excel的格式存储。最广泛使用的是SQL-based的关系型数据库（SQL Server，PostgreSQL，MySQL）。选择数据库通常取决于性能，数据整合性，实际应用的可扩展性。

读取SQL到DataFrame非常直观，pandas中有一些函数能简化这个过程。举个例子，这里创建一个SQLite数据库，通过使用python内建的sqlite3 driver：

In [1]:
import sqlite3
import pandas as pd

In [5]:
query = """
CREATE TABLE test2
(
    a VARCHAR(20), 
    b VARCHAR(20),
    c REAL,        
    d INTEGER
);"""

In [17]:
query = """
Create Table uniswap_swap
(
    pair_name VARCHAR(20),
    transaction_hash VARCHAR(255),
    block_timestamp datetime,
    block_number    INTEGER,
    log_index       INTEGER,
    contract_address    VARCHAR(255),
    sender_addr              VARCHAR(255),
    amount0In           REAL,
    amount0Out          REAL,
    amount1In           REAL,
    amount1Out          REAL,
    to_addr                  VARCHAR(255),
    actual_price_a2b    REAL,
    actual_price_b2a    REAL
);

"""

In [22]:
con = sqlite3.connect('../uniswap.sqlite')

In [23]:
con.execute(query)

In [24]:
con.commit()

然后我们插入几行数据：

In [7]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [8]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [9]:
con.executemany(stmt, data)

In [10]:
con.commit()

大部分python的SQL驱动(PyODBC, psycopg2, MySQLdb, pymssql, 等)返回a list of tuple，当从一个表格选择数据的时候：

In [11]:
cursor = con.execute('select * from test')

In [12]:
rows = cursor.fetchall()

In [13]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

我们可以把list of tuples传递给DataFrame，但是我们也需要column names,包含cursor的description属性：

In [14]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [16]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


我们不希望每次询问数据库的时候都重复以上步骤，这样对计算机很不好(mung,【计】逐步对计算机系统或文件做小改动导致大的损害)。SQLAlchemy计划是一个六星的Python SQL工具箱，它能抽象出不同SQL数据库之间的不同。pandas有一个read_sql函数，能让我们从SQLAlchemy connection从读取数据。这里我们用SQLAlchemy连接到同一个SQLite数据库，并从之前创建的表格读取数据：

In [17]:
import sqlalchemy as sqla

In [21]:
db = sqla.create_engine('sqlite:///../examples/mydata.sqlite')

In [22]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
